### Machine limits for integer and floating-point types
Get the machine limit for <i>np.float32</i>, <i>np.float64</i>, and <i>np.double</i> using <i>np.iinfo()</i>.

In [ ]:
import numpy as np
np.iinfo(np.int32)

In [ ]:
np.iinfo(np.int32).max

In [ ]:
np.iinfo(int)

In [ ]:
np.finfo(np.float32)

In [ ]:
np.finfo(np.float32).eps

In [ ]:
np.finfo(np.float64)

In [ ]:
np.finfo(np.double)

### Inexactness
Are the following expresions <i>True</i>?

In [ ]:
0.1 + 0.1 + 0.1 == 0.3

In [ ]:
x1 = 0.1
x2 = 0.10000000000000001
x3 = 0.1000000000000000055511151231257827021181583404541015625

In [ ]:
eval(repr(x1)) == x1

In [ ]:
eval(repr(x1)) == x2

In [ ]:
eval(repr(x1)) == x3

### Holes in value range
What is the result of the following subtraction? Is it 0.0?

In [ ]:
a = 1.0
b = 0.1
c = 1.1
c - a - b

### Conversions

In [ ]:
import numpy as np
np.__version__

Conversions to an integer can reveal the inaccuracies in a floating-point number. The closest *single-precision* floating-point number to 20.23 is slightly less than 20.23. When it is multiplied by a hundred, the result is slightly less than 2023.0. Note, there is no rounding in converting 'y' to an integer 'i', the number is truncated:

In [ ]:
x = np.float32(20.23)
y = x * 100.
i = int(y)
print(i, y)

In [ ]:
x = np.float64(20.23)
y = x * 100.
i = int(y)
print(i, y)

Assigning a single-precision number to a double-precision number doesn't increase the number of significant digits:

In [ ]:
x = np.float32(1.66661)
y = np.float64(x)
print(y)

Why there are simingly random <i>00025177002</i> numbers and not <i>00000000000</i>?

The floating-point padding with zeros is done in the binary representation:
1.10101010101001101111010000000000000000000000000000000000010101...

### Rounding

In [ ]:
round(256.49999) == 256

In [ ]:
-1.225 * 100

### Decimal fixed point and floating-point arithmetic
https://docs.python.org/3/library/decimal.html

In [ ]:
from decimal import *
getcontext()

In [ ]:
getcontext().prec = 6

In [ ]:
Decimal(0.1) + Decimal(0.1) + Decimal(0.1)

### Accuracy of floating-point arithmetic
Examples from Donald E. Knuth The Art of Computer Programming, volume 2 / Seminumerical Algorithms, Section 4.2.2

In [ ]:
from decimal import Decimal, getcontext
getcontext().prec = 8

u, v, w = Decimal(11111113), Decimal(-11111111), Decimal('7.51111111')
(u + v) + w

In [ ]:
u + (v + w)

In [ ]:
u, v, w = Decimal(20000), Decimal(-6), Decimal('6.0000003')
(u*v) + (u*w)

In [ ]:
u * (v+w)

In [ ]:
u, v, w = Decimal(8.0000001), Decimal(1.2500008), Decimal(8.0000008)
(u * v) * w

In [ ]:
u * (v * w)

### Summing many numbers

In [ ]:
import numpy as np

def summ():
    tenth = np.float32(0.1)
    count = np.float32(60*60*100*10)
    print(f"{count} {count*0.1}")
    sum = np.float32(0)
    n = np.int64(0)
    while n < 1000000:
        sum += tenth
        n += 1
        if n < 21 or n%36000 == 0:
            print(f"step {n} expected {0.1*n} solution {sum} diff {np.abs(0.1*n - sum)}")

summ()

### Kahan summation algorithm
https://en.wikipedia.org/wiki/Kahan_summation_algorithm

In [ ]:
import numpy as np

def kahan_summ():
    tenth = np.float32(0.1)
    count = np.float32(60*60*100*10)
    print(f"{count} {count*0.1}")
    sum = np.float32(0)
    n = np.int64(0)
    c = np.float32(0)
    while n < 1000000:
        y = tenth - c
        x = sum + y
        c = (x - sum) - y
        sum = x
        n += 1
        if n < 21 or n%36000 == 0:
            print(f"step {n} expected {0.1*n} solution {sum} diff {np.abs(0.1*n - sum)}")

kahan_summ()

### Plot Example

In [ ]:
import numpy as np

def plot_summ(k):
    s = np.float32(0)
    for i in range(k - 1):
        s += np.float32(0.1)
    return abs(k/10 - s)

for i in range(100000):
    if i%3600 == 0:
        print(plot_summ(i))

### Floating-Point Numbers Don't-s:
#### 1. Test rounded floating-point numbers for equality 

Example: this is an endless loop because x never becomes exactly 1:

In [ ]:
def endless_loop():
    x = 0.1
    while x != 1.0:
        x += 0.1
        print(x)

#### 2. Add two floating-point numbers of very different orders of magnitude

The following function calculates the n-th [harmonic number](https://en.wikipedia.org/wiki/Harmonic_number) in two ways. What is the difference between the two loops? One loop runs through numbers 1 up to n, adding reciprocals. Second loop runs from n down to 1 and summs up the recipricals. Does the order matter?

In [ ]:
import numpy as np

def harmonic_number(n):
    # forward sum
    f_sum = np.float32(0.0)
    i = np.int32(1)
    while i <= n:
        f_sum += np.float32(1.0/i)
        i += 1
    
    # backward sum
    b_sum = np.float32(0.0)
    i = np.int32(n)
    while i >= 1:
        b_sum += np.float32(1.0/i)
        i -= 1

    print("Forward sum", f_sum)
    print("Backward sum", b_sum)

In [ ]:
harmonic_number(10000000)

In [ ]:
harmonic_number(100000000)

#### 3. Subtract two floating-point numbers with very similar value

The most significant digits cancel each other in subtracting two numbers that are almost equal. If, on the other hand, the remaining less significant digits  already carry some errors fom previous computations, the subtraction hugely amplifies these errors: the cancellation promotes the previously less significant digits to much more significant digis in the result. 

In [ ]:
import numpy as np

def func(x):
    t = np.exp(-np.pi * x)
    return 1/t*(1-np.sqrt(1-t**2))

### Cancellation

Approximaion of Pi

In [ ]:
import numpy as np

x = np.float32(3.141592653589793)
y = np.float32(3.141592653585682)

In [ ]:
z = x - y
z

In [ ]:
import numpy as np

x = np.float64(3.141592653589793)
y = np.float64(3.141592653585682)

In [ ]:
z = x - y
z

Quadratic equations

In [ ]:
a=np.float64(1.0)
b=np.float64(1.786737589984535)
c=np.float64(1.149782767465722e-8)

In [ ]:
x_1 = (-b - np.sqrt(b**2 - 4*a*c))/(2*a)
x_2 = (-b + np.sqrt(b**2 - 4*a*c))/(2*a)
print(x_1, x_2)

In [ ]:
x_1 = (-b - np.sign(b)*np.sqrt(b**2 - 4*a*c))/(2*a)
x_2 = c/(a*x_1)
print(x_1, x_2)

### The Sign Bit

IEEE 754-2019: Except that <b>squareRoot(-1)</b> shall be -0, every numeric <b>squareRoot</b> result shall have a positive sign.

In [ ]:
import math
math.sqrt(-0.0)

### Solar System

In [ ]:
# solarsystem.py

import itertools
import math
import turtle
import numpy as np


# Solar System Bodies
class SolarSystemBody(turtle.Turtle):
    min_display_size = 20
    display_log_base = 1.1

    def __init__(
            self,
            solar_system,
            mass,
            position=(0, 0),
            velocity=(0, 0),
    ):
        super().__init__()
        self.mass = mass
        self.setposition(position)
        self.velocity = velocity
        self.display_size = max(
            math.log(self.mass, self.display_log_base),
            self.min_display_size,
        )

        self.penup()
        self.hideturtle()

        solar_system.add_body(self)

    def draw(self):
        self.clear()
        self.dot(self.display_size)

    def move(self):
        self.setx(self.xcor() + self.velocity[0])
        self.sety(self.ycor() + self.velocity[1])


class Sun(SolarSystemBody):
    def __init__(
            self,
            solar_system,
            mass,
            position=(0, 0),
            velocity=(0, 0),
    ):
        super().__init__(solar_system, mass, position, velocity)
        self.color("yellow")



class Planet(SolarSystemBody):
    colours = itertools.cycle(["red", "green", "blue"])

    def __init__(
            self,
            solar_system,
            mass,
            position=(0, 0),
            velocity=(0, 0),
    ):
        super().__init__(solar_system, mass, position, velocity)
        self.color(next(Planet.colours))


# Solar System
class SolarSystem:
    def __init__(self, width, height):
        self.solar_system = turtle.Screen()
        self.solar_system.tracer(0)
        self.solar_system.setup(width, height)
        self.solar_system.bgcolor("black")

        self.bodies = []

    def add_body(self, body):
        self.bodies.append(body)

    def remove_body(self, body):
        self.bodies.remove(body)

    def update_all(self):
        for body in self.bodies:
            body.move()
            body.draw()
        self.solar_system.update()
        
    @staticmethod
    def accelerate_due_to_gravity(
            first: SolarSystemBody,
            second: SolarSystemBody,
    ):
        force = first.mass * second.mass / first.distance(second) ** 2
        angle = first.towards(second)
        reverse = 1
        for body in first, second:
            acceleration = force / body.mass
            acc_x = acceleration * math.cos(math.radians(angle))
            acc_y = acceleration * math.sin(math.radians(angle))
            body.velocity = (
                body.velocity[0] + (reverse * acc_x),
                body.velocity[1] + (reverse * acc_y),
            )
            reverse = -1


In [ ]:
# simple_solar_system.py

#from solarsystem import SolarSystem, Sun, Planet

solar_system = SolarSystem(width=1400, height=900)

sun = Sun(solar_system, mass=10_000)
planet = Planet(
    solar_system,
    mass=1,
    position=(-350, 0),
    velocity=(0, 5),
)

while True:
    solar_system.accelerate_due_to_gravity(sun, planet)
    solar_system.update_all()
    